<a href="https://colab.research.google.com/github/Kevin2558/Data_Science_Borrador/blob/main/04_Modelos_Supervisados/Regresion/Ridge_Lasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ridge - Lasso

Estos modelos son extensiones de la regresión lineal que incorporan técnicas de regularización para mejorar la capacidad de generalización del modelo y así evitar el sobreajuste.

Ambas buscan minimizar el error del modelo añadiendo una penalización al tamaño de los coeficientes:

- Ridge: Utiliza una penalización L2 que reduce los coeficientes sin hacerlos cero.

- Lasso: Emplea una penalización L1 que puede forzar algunos coeficientes a cero, funcionando también como método de selección de variables.

Estas técnicas son especialmente útiles en contextos con muchas variables predictoras o cuando existe multicolinealidad entre ellas.

Carguemos el dataset a utilizar.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("harlfoxem/housesalesprediction")

print("Path to dataset files:", path)

100%|██████████| 780k/780k [00:00<00:00, 16.4MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/harlfoxem/housesalesprediction/versions/1


Visualicemos el dataset.

In [ ]:
import pandas as pd

df = pd.read_csv( path + "/kc_house_data.csv")

In [ ]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


Carguemos las librerías a utilizar.

In [ ]:
from sklearn.compose   import ColumnTransformer
from sklearn.pipeline  import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model  import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

Separemos entre variables numéricas y categóricas.

In [ ]:
num = ["sqft_living", "bathrooms","lat","long"]
cat = ["zipcode","waterfront"]

In [ ]:
X = df[num+cat] # + concatena las columnas
y = df["price"] # Variable objetivo

División de los datos entre entrenamiento y validación.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

Preprocesamiento.

In [ ]:
pre = ColumnTransformer([
    ("num", Pipeline(
        [
            ("scaler", StandardScaler()),
            ("pca", PCA(n_components=0.95))
        ]
    ),num),
    ("cat", OneHotEncoder(drop="first"), cat)
])

Partiremos realizando el modelo solo con regresión lineal.

Lo que haremos en el pipeline es hacer el preprocesamiento de los datos y luego realizaremos la regresion lineal.

Entrenemos el modelo.

In [ ]:
lin = Pipeline([
    ("pre", pre),
    ("lin", LinearRegression())
])

# Entrenamiento

lin.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=0.95))]),
                                                  ['sqft_living', 'bathrooms',
                                                   'lat', 'long']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['zipcode', 'waterfront'])])),
                ('lin', LinearRegression())])

Continuemos con la predicción del modelo.

In [ ]:
pred = lin.predict(X_test)

In [ ]:
pred

array([ 358777.2611405 ,  876149.52663115, 1074621.66525694, ...,
        407594.24810531,  639749.63663792,  437996.72105466])

Definimos la metrica de la raíz del error cuadratico medio para evaluar las predicciones del modelo.

In [ ]:
def rmse(a,b): return np.sqrt(mean_squared_error(a,b))

In [ ]:
# Error medio absoluto
print("MAE:", mean_absolute_error(y_test, pred))

# Raiz del error cuadratico medio
print("RMSE:", rmse(y_test, pred))

print("R²:", r2_score(y_test, pred))

MAE: 109653.70184747483
RMSE: 185420.75844868482
R²: 0.7725783478789328


Lo que buscamos es que MAE sea mas pequeño y R² sea cercano a 1, así nuestra predicción será mejor.

Notemos que si aumentamos el número de componentes los errores mejoraran, debido a que tendremos mas variables que expliquen la dispersion de los datos.

Ahora veremos como mejorar nuestra predicción a través de los métodos de regularizacion Ridge y Lasso.

In [ ]:
from sklearn.linear_model import Ridge, Lasso

Con el siguiente ciclo for lo que haremos es que se ejecute un método y luego el otro para poder compararlos.

In [ ]:
for Model, name in [(Ridge(alpha=1.0), "Ridge"),
                    (Lasso(alpha=0.1),  "Lasso")]:
    mdl = Pipeline([("prep", pre), ("reg", Model)])
    mdl.fit(X_train, y_train)
    y_pred = mdl.predict(X_test)
    print(name,
          "RMSE:", rmse(y_test,y_pred),
          "R²:",  r2_score(y_test,y_pred))

Ridge RMSE: 185608.55181278766 R²: 0.7721174512149256
Lasso RMSE: 185420.35548674664 R²: 0.7725793363569318


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 222384952374006.22, tolerance: 225891453177.81256
  model = cd_fast.sparse_enet_coordinate_descent(


Notemos que el método Ridge no mejoró el R^2 y Lasso lo mejoró pero no significativamente.

En este caso, o podemos utilizar otros valores de alpha en los metodos o podemos ver si las variables que estamos utilizando para la prediccion son realmente relevante sino cambiarlas por otras que lo sean mas.

Ahora, realicemos el preprocesamiento con reducción de dimensionalidad PCA con 95% de la varianza.

In [ ]:
pre = ColumnTransformer([
    ("num", Pipeline(
        [
            ("scaler", StandardScaler()),
            ("pca", PCA(n_components=0.95))
        ]
    ),num),
    ("cat", OneHotEncoder(drop="first"), cat)
])

Veamos cuantas variables quedaron luego de la transformación.

In [ ]:
X_tr_tans = pre.fit_transform(X_train)
print("Shape final:", X_tr_tans.shape)

Shape final: (17290, 74)


Con lo siguiente llegaremos al paso del PCA luego de realizado el StandardScaler.

In [ ]:
pca_model = lin.named_steps["pre"].named_transformers_["num"].named_steps["pca"]

In [ ]:
pca_model.n_components_

np.int64(4)

Quedaron 4 componentes principales por lo que no se hizo una reduccion.

Con lo siguiente llegaremos al paso luego del OneHotEncoder.

In [ ]:
encoder = lin.named_steps["pre"].named_transformers_["cat"]
print("Categorias modificadas:", encoder.get_feature_names_out())

Categorias modificadas: ['zipcode_98002' 'zipcode_98003' 'zipcode_98004' 'zipcode_98005'
 'zipcode_98006' 'zipcode_98007' 'zipcode_98008' 'zipcode_98010'
 'zipcode_98011' 'zipcode_98014' 'zipcode_98019' 'zipcode_98022'
 'zipcode_98023' 'zipcode_98024' 'zipcode_98027' 'zipcode_98028'
 'zipcode_98029' 'zipcode_98030' 'zipcode_98031' 'zipcode_98032'
 'zipcode_98033' 'zipcode_98034' 'zipcode_98038' 'zipcode_98039'
 'zipcode_98040' 'zipcode_98042' 'zipcode_98045' 'zipcode_98052'
 'zipcode_98053' 'zipcode_98055' 'zipcode_98056' 'zipcode_98058'
 'zipcode_98059' 'zipcode_98065' 'zipcode_98070' 'zipcode_98072'
 'zipcode_98074' 'zipcode_98075' 'zipcode_98077' 'zipcode_98092'
 'zipcode_98102' 'zipcode_98103' 'zipcode_98105' 'zipcode_98106'
 'zipcode_98107' 'zipcode_98108' 'zipcode_98109' 'zipcode_98112'
 'zipcode_98115' 'zipcode_98116' 'zipcode_98117' 'zipcode_98118'
 'zipcode_98119' 'zipcode_98122' 'zipcode_98125' 'zipcode_98126'
 'zipcode_98133' 'zipcode_98136' 'zipcode_98144' 'zipcode_98146'
 

Notemos que como la variable zipcode poseia muchos valores, se crearon demasiadas variables dummy para representarla.

Dado que las 4 componentes principales no hicieron realmente mucho, vamos a agregar mas variables numéricas y categoricas para ver si mejora la predicción.

In [ ]:
num = ["sqft_living", "bathrooms","lat","long","sqft_lot","bedrooms","yr_built"]
cat = ["zipcode","waterfront","condition","grade"]

Realicemos todo el proceso nuevamente.

In [ ]:
X = df[num+cat]
y = df["price"]

X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.2,random_state=42)

pre = ColumnTransformer([
    ("num", Pipeline([("sc",StandardScaler()),
                      ("pca",PCA(n_components=0.95))]), num),
    ("cat", OneHotEncoder(drop="first"), cat)
])

lin = Pipeline([("prep",pre),("lr",LinearRegression())])
lin.fit(X_tr,y_tr)
pred = lin.predict(X_te)

def rmse(a,b): return np.sqrt(mean_squared_error(a,b))
print("MAE :", mean_absolute_error(y_te,pred))
print("RMSE:", rmse(y_te,pred))
print("R²  :", r2_score(y_te,pred))

MAE : 96263.78618602717
RMSE: 165624.37999860596
R²  : 0.8185472164302972


Notemos que el MAE bajó entorno a 10000 y el R^2 se acerca aún más 1, por lo que al agregar variables mejoramos la predicción.

Ahora, como se habia dicho antes, la variable zipcode genera demasiadas variables dummy, por lo que podriamos considerar quitar esta variable y ver que tal nos resulta la prediccion.

Además notemos que en num tenemos las variables lat y long las cuales pueden llegar a representar zipcode cuando las casas se ubican considerablemente lejos.


In [ ]:
num = ["sqft_living", "bathrooms","lat","long","sqft_lot","bedrooms","yr_built"]
cat = ["waterfront","condition","grade"]

In [ ]:
X = df[num+cat]
y = df["price"]

X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.2,random_state=42)

pre = ColumnTransformer([
    ("num", Pipeline([("sc",StandardScaler()),
                      ("pca",PCA(n_components=0.95))]), num),
    ("cat", OneHotEncoder(drop="first"), cat)
])

lin = Pipeline([("prep",pre),("lr",LinearRegression())])
lin.fit(X_tr,y_tr)
pred = lin.predict(X_te)

def rmse(a,b): return np.sqrt(mean_squared_error(a,b))
print("MAE :", mean_absolute_error(y_te,pred))
print("RMSE:", rmse(y_te,pred))
print("R²  :", r2_score(y_te,pred))

MAE : 125883.5445446909
RMSE: 206637.9602770373
R²  : 0.7175540730446212


Notemos que el error aumentó considerablemente y el R^2 bajó demasiado, por lo que la variable zipcode es realmente relevante para predecir el precio de una propiedad.

Ahora lo que haremos es volver a nuestras variables con zipcode y aplicaremos Ridge y Lasso para ver si las predicciones mejoran.


In [ ]:
num = ["sqft_living", "bathrooms","lat","long","sqft_lot","bedrooms","yr_built"]
cat = ["zipcode","waterfront","condition","grade"]

In [ ]:
X = df[num+cat]
y = df["price"]

X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.2,random_state=42)

pre = ColumnTransformer([
    ("num", Pipeline([("sc",StandardScaler()),
                      ("pca",PCA(n_components=0.95))]), num),
    ("cat", OneHotEncoder(drop="first"), cat)
])

lin = Pipeline([("prep",pre),("lr",LinearRegression())])
lin.fit(X_tr,y_tr)
pred = lin.predict(X_te)

def rmse(a,b): return np.sqrt(mean_squared_error(a,b))
print("MAE :", mean_absolute_error(y_te,pred))
print("RMSE:", rmse(y_te,pred))
print("R²  :", r2_score(y_te,pred))

MAE : 96263.78618602717
RMSE: 165624.37999860596
R²  : 0.8185472164302972


In [ ]:
for Model, name in [(Ridge(alpha=1.0), "Ridge"),
                    (Lasso(alpha=0.1, max_iter=100000),  "Lasso")]:
    mdl = Pipeline([("prep", pre), ("reg", Model)])
    mdl.fit(X_tr, y_tr)
    y_pred = mdl.predict(X_te)
    print(name,
          "RMSE:", rmse(y_te,y_pred),
          "R²:",  r2_score(y_te,y_pred))

Ridge RMSE: 165830.37636382855 R²: 0.8180955696519866
Lasso RMSE: 165624.932824597 R²: 0.8185460051112594


Para los alphas dados no se mejoró la predicción por lo que podemos variar sus valores para poder obtener mejores predicciones.

In [ ]:
for Model, name in [(Ridge(alpha=0.1), "Ridge 0.1"),
                    (Ridge(alpha=0.01), "Ridge 0.01"),
                    (Ridge(alpha=10.0), "Ridge 10.0"),
                    (Lasso(alpha=1,max_iter=10000),  "Lasso 1"),
                    (Lasso(alpha=10,max_iter=10000),  "Lasso 10"),
                    (Lasso(alpha=100,max_iter=10000),  "Lasso 100"),
                    ]:
    mdl = Pipeline([("prep", pre), ("reg", Model)])
    mdl.fit(X_tr, y_tr)
    y_pred = mdl.predict(X_te)
    print(name,
          "RMSE:", rmse(y_te,y_pred),
          "R²:",  r2_score(y_te,y_pred))

Ridge 0.1 RMSE: 165628.78705245885 R²: 0.8185375598467909
Ridge 0.01 RMSE: 165617.42363632304 R²: 0.8185624584471062
Ridge 10.0 RMSE: 169034.13926415017 R²: 0.8109990632140089
Lasso 1 RMSE: 165632.2917588195 R²: 0.8185298802723867
Lasso 10 RMSE: 165713.94004867197 R²: 0.818350925109437
Lasso 100 RMSE: 166304.44080873916 R²: 0.817054051382487


El único que mejoró, aunque no considerablemente, fue Ridge con alpha=0.01.

Por último, veremos las variables que explican de mejor manera la prediccion del precio.

Obtendremos los coeficientes de la regresión lineal.

El mayor coeficiente será la variable que más incide en la prediccion de la variable objetivo (en este caso el precio de una propiedad).

In [ ]:
beta = lin.named_steps['lr'].coef_
feature_names = lin.named_steps['prep'].get_feature_names_out(num + cat)
pd.Series(beta, index=feature_names).sort_values(key=abs, ascending=False).head(10)

,0
cat__grade_13,2.105769e+06
cat__zipcode_98039,1.127695e+06
cat__grade_12,8.703386e+05
cat__waterfront_1,7.887491e+05
cat__zipcode_98004,6.629506e+05
cat__zipcode_98112,4.938669e+05
cat__zipcode_98040,4.565285e+05
cat__grade_11,4.518589e+05
cat__zipcode_98109,3.964299e+05
cat__zipcode_98119,3.732756e+05


Por ejemplo, cat_grade_13 explica 210000 dolares entre un grado y otro.

Principalmente, son las variables categóricas las que están explicando más el valor de la predicción del precio.